In [2]:
import os
import uuid
from tqdm.auto import tqdm
from typing import List
from dotenv import load_dotenv
from qdrant_client import QdrantClient
from qdrant_client.http.models import PointStruct
from fastembed.sparse.bm25 import Bm25
from fastembed.late_interaction import LateInteractionTextEmbedding
from fastembed import TextEmbedding
from fastembed.sparse.bm25 import Bm25

import json
from transformers import AutoTokenizer, AutoModelForTokenClassification, pipeline


/home/giuseppe/projetos-pessoais/test-ai-engineer/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# dense_embedding_model = TextEmbedding(
#     "sentence-transformers/paraphrase-multilingual-mpnet-base-v2"
# )
# bm25_embedding_model = Bm25("Qdrant/bm25")

In [3]:
import os
import uuid
from tqdm.auto import tqdm
from typing import List
from dotenv import load_dotenv
from qdrant_client import QdrantClient
from qdrant_client.http.models import PointStruct
from fastembed.sparse.bm25 import Bm25
from fastembed.late_interaction import LateInteractionTextEmbedding
from fastembed import TextEmbedding

from transformers import AutoTokenizer, AutoModelForTokenClassification, pipeline
from docling.chunking import HybridChunker
from docling.document_converter import DocumentConverter


# Constants
PDF_PATH = "./D23569.pdf"
EMBED_MODEL_ID = "sentence-transformers/paraphrase-multilingual-mpnet-base-v2"
MAX_TOKENS = 750
NER_MODEL_NAME = "nickprock/bert-italian-finetuned-ner"  # Italian  NER model
MIN_ENTITY_CONFIDENCE = 0.80  # Minimum confidence threshold for entity extraction


def convert_pdf_to_document(pdf_path):
    """
    Convert a PDF file to a structured document format.
    """
    converter = DocumentConverter()
    result = converter.convert(pdf_path)
    return result.document


def create_document_chunks(document, embed_model_id, max_tokens):
    """
    Split the document into manageable chunks for processing.
    """
    tokenizer = AutoTokenizer.from_pretrained(embed_model_id)

    chunker = HybridChunker(
        tokenizer=tokenizer,
        max_tokens=max_tokens,
        merge_peers=True,
    )

    chunk_iter = chunker.chunk(dl_doc=document)
    return list(chunk_iter)


def setup_ner_pipeline(model_name):
    """
    Set up a Named Entity Recognition pipeline.
    """
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModelForTokenClassification.from_pretrained(model_name)

    return pipeline(
        "ner", model=model, tokenizer=tokenizer, aggregation_strategy="first"
    )


def extract_entities_from_chunk(chunk_text, ner_pipeline, min_confidence=0.75):
    """
    Extract named entities from a text chunk with quality filtering.

    Args:
        chunk_text (str): The text to extract entities from
        ner_pipeline: The NER pipeline to use
        min_confidence (float): Minimum confidence threshold

    Returns:
        dict: Dictionary of entity types and their values
    """
    # Limit text size to prevent performance issues

    # Extract entities using the NER pipeline
    raw_entities = ner_pipeline(chunk_text)

    # Filter entities by confidence and length
    filtered_entities = {}

    for entity in raw_entities:
        # Skip very short tokens (unless they're acronyms)
        if len(entity["word"]) < 2 and not entity["word"].isupper():
            continue

        # Filter by confidence score
        if entity.get("score", 0) < min_confidence:
            continue

        entity_type = entity["entity_group"]

        if entity_type not in filtered_entities:
            filtered_entities[entity_type] = set()

        filtered_entities[entity_type].add(entity["word"])

    # Convert sets to lists for better serialization
    for entity_type in filtered_entities:
        filtered_entities[entity_type] = list(filtered_entities[entity_type])

    return filtered_entities


def enrich_document_with_metadata(text, ner_pipeline, name=None):
    """
    Enrich a documents text with metadata such as page numbers and headings,
    and extract named entities using a NER pipeline.
    """

    # Basic metadata
    metadata = {
        "page_numbers": getattr(text.meta, "page_numbers", None)
        if hasattr(text, "meta")
        else None,
    }
    metadata["name"] = name if name else None

    # Add headings if available
    if hasattr(text, "meta") and hasattr(text.meta, "headings"):
        metadata["headings"] = text.meta.headings

    try:
        entities = extract_entities_from_chunk(
            text, ner_pipeline, MIN_ENTITY_CONFIDENCE
        )
        if entities:
            metadata["entities"] = entities
    except Exception as e:
        metadata["entities_error"] = str(e)

    return {"text": text, "metadata": metadata}


def initialize_embedding_models():
    """
    Initialize the three embedding models needed for hybrid search:
    """
    dense_embedding_model = TextEmbedding(
        "sentence-transformers/paraphrase-multilingual-mpnet-base-v2"
    )
    bm25_embedding_model = Bm25("Qdrant/bm25")

    colbert_embedding_model = LateInteractionTextEmbedding("colbert-ir/colbertv2.0")

    return dense_embedding_model, bm25_embedding_model, colbert_embedding_model


def create_embeddings(chunk_text, dense_model, bm25_model, colbert_model):
    """
    Create the three types of embeddings for a text chunk.
    """
    # Generate embeddings for each model
    dense_embedding = list(dense_model.passage_embed([chunk_text]))[0].tolist()
    sparse_embedding = list(bm25_model.passage_embed([chunk_text]))[0].as_object()
    colbert_embedding = list(colbert_model.passage_embed([chunk_text]))[0].tolist()

    return {
        "dense": dense_embedding,
        "sparse": sparse_embedding,
        "colbertv2.0": colbert_embedding,
    }


def prepare_point(data_text: dict, embedding_models):
    """
    Prepare a single data point for Qdrant ingestion.
    """
    dense_model, bm25_model, colbert_model = embedding_models

    # Extract text from chunk based on your structure
    text = data_text.get("text", "")

    # Create embeddings
    embeddings = create_embeddings(text, dense_model, bm25_model, colbert_model)

    # Prepare payload with metadata from chunk
    payload = {"text": text, "metadata": data_text.get("metadata", {})}

    # Create and return the point
    return PointStruct(
        id=str(uuid.uuid4()),
        vector={
            "dense": embeddings["dense"],
            "sparse": embeddings["sparse"],
            "colbertv2.0": embeddings["colbertv2.0"],
        },
        payload=payload,
    )


def upload_in_batches(
    client: QdrantClient,
    collection_name: str,
    points: List[PointStruct],
    batch_size: int = 10,
):
    """
    Upload points to Qdrant in batches with progress tracking.
    If only one point is provided, upload it directly.
    """
    print(f"Uploading {len(points)} point(s) to collection '{collection_name}'")

    if len(points) == 1:
        client.upload_points(collection_name=collection_name, points=points)
    else:
        for i in tqdm(range(0, len(points), batch_size), desc="Uploading batches"):
            batch = points[i : i + batch_size]
            client.upload_points(collection_name=collection_name, points=batch)

    print(
        f"Successfully uploaded {len(points)} point(s) to collection '{collection_name}'"
    )


def process_and_upload_chunks(text, collection_name):
    """
    Process document chunks and upload them to Qdrant.
    """
    # Load environment variables
    load_dotenv()

    # Initialize client
    client = QdrantClient(
        url=os.getenv("QDRANT_URL"),
    )

    # Initialize embedding models
    embedding_models = initialize_embedding_models()

    # Prepare points
    print("Preparing points with embeddings...")

    point = prepare_point(text, embedding_models)



    # Upload points in batches
    upload_in_batches(
        client=client,
        collection_name=collection_name,
        points=[point],  # Wrap in a list for single point upload
        batch_size=1,  # Adjust based on your document size and memory constraints
    )

    # Print confirmation with collection info
    collection_info = client.get_collection(collection_name)
    print(
        f"Collection '{collection_name}' now has {collection_info.points_count} points"
    )


In [4]:
# with open("../data/gold/technic_specification.json", "r", encoding="utf-8") as f:
#     information = json.load(f)

In [5]:
# information

In [6]:
# import os
# from dotenv import load_dotenv
# from qdrant_client import QdrantClient, models
# from qdrant_client.http.models import VectorParams, Distance

# # Load environment variables
# load_dotenv(override=True)


# # configurations for the collections

# for COLLECTION_NAME in ["bandi", "fornitori"]:
#     # initialize Qdrant client
#     client = QdrantClient(
#         url=os.getenv("QDRANT_URL"),
#         api_key=os.getenv("QDRANT_API_KEY"),
#     )

#     # check if the collection already exists
#     collections = client.get_collections().collections
#     collection_names = [collection.name for collection in collections]
#     if COLLECTION_NAME in collection_names:
#         print(f"Collection '{COLLECTION_NAME}' already exists. Deleting it...")
#         client.delete_collection(COLLECTION_NAME)

#     # Create the collection with configuration for hybrid search
#     client.create_collection(
#         collection_name=COLLECTION_NAME,
#         vectors_config={
#             # Dense vector (semantic)
#             "dense": VectorParams(size=768, distance=Distance.COSINE),
#             # Late interaction vector (ColBERT)
#             "colbertv2.0": VectorParams(
#                 size=128,
#                 distance=Distance.COSINE,
#                 multivector_config=models.MultiVectorConfig(
#                     comparator=models.MultiVectorComparator.MAX_SIM,
#                 )
#             )
#         },
#         sparse_vectors_config={
#             # Sparse vector for BM25 (bag-of-words)
#             "sparse": models.SparseVectorParams(modifier=models.Modifier.IDF),
#         },
#     )

#     print(f"Collection '{COLLECTION_NAME}' created successfully!")

#     # show collection information
#     collection_info = client.get_collection(COLLECTION_NAME)
#     print(f"Status: {collection_info.status}")
#     print(f"Configured vectors: {list(collection_info.config.params.vectors.keys())}")
#     print(
#         f"Sparse vectors: {list(collection_info.config.params.sparse_vectors.keys() if collection_info.config.params.sparse_vectors else [])}"
#     )
#     print(f"Points: {collection_info.points_count}")
#     print("\n" + "=" * 50 + "\n")


In [7]:
# # Set up NER pipeline
# ner_pipeline = setup_ner_pipeline(NER_MODEL_NAME)

# for key in information:
#     print(f"Processing {key}...")

#     # Process chunks and extract metadata
#     enriched_chunks = enrich_document_with_metadata(
#         information[key], ner_pipeline, name=key
#     )

#     # Send data to Qdrant
#     test = process_and_upload_chunks(enriched_chunks, "bandi")

In [8]:
with open(
    "../data/gold/fornitori/fornitori_description.json", "r", encoding="utf-8"
) as f:
    information_fornitori = json.load(f)

In [9]:
information_fornitori

{'Gestione Reti Lombardia S.r.l.': 'Il servizio consiste nella manutenzione preventiva e correttiva delle dorsali radio e fibra ottica regionali, con help‑desk H24, drive test, tuning di copertura, gestione ticket e advance replacement di apparati, migrazione verso nuovi sistemi e monitoraggio KPI dei flussi dati verso le piattaforme centrali.',
 'Assistenza Tecnica Puglia S.c.p.A.': 'Fornisce supporto specialistico all’Autorità di Gestione per la programmazione e il monitoraggio di programmi operativi complementari, incluse attività di rendicontazione POR, riprogrammazione POC, controlli on‑site conformi MEF‑IGRUE, definizione specifiche BeS e redazione della relazione annuale di attuazione.',
 'BioCucina Italiana S.r.l.': 'Produce e distribuisce alimenti biologici a km zero; offre servizio catering per eventi privati e aziendali, delivery di box stagionali, consulenza per riduzione degli sprechi alimentari e progetti di educazione nutrizionale nelle scuole.',
 'Viaggi e Scoperte S.p.

In [ ]:
# Set up NER pipeline
ner_pipeline = setup_ner_pipeline(NER_MODEL_NAME)

for key in information_fornitori:
    print(f"Processing {key}...")

    # Process chunks and extract metadata
    enriched_chunks = enrich_document_with_metadata(
        information_fornitori[key], ner_pipeline, name=key
    )

    # Send data to Qdrant
    test = process_and_upload_chunks(enriched_chunks, "fornitori")

Device set to use cpu


Processing Gestione Reti Lombardia S.r.l....


/tmp/ipykernel_31653/1898059344.py:139: UserWarning: The model sentence-transformers/paraphrase-multilingual-mpnet-base-v2 now uses mean pooling instead of CLS embedding. In order to preserve the previous behaviour, consider either pinning fastembed version to 0.5.1 or using `add_custom_model` functionality.
  dense_embedding_model = TextEmbedding(


Preparing points with embeddings...
Uploading 1 point(s) to collection 'fornitori'
Successfully uploaded 1 point(s) to collection 'fornitori'
Collection 'fornitori' now has 1 points
Processing Assistenza Tecnica Puglia S.c.p.A....
Preparing points with embeddings...
Uploading 1 point(s) to collection 'fornitori'
Successfully uploaded 1 point(s) to collection 'fornitori'
Collection 'fornitori' now has 2 points
Processing BioCucina Italiana S.r.l....
Preparing points with embeddings...
Uploading 1 point(s) to collection 'fornitori'
Successfully uploaded 1 point(s) to collection 'fornitori'
Collection 'fornitori' now has 3 points
Processing Viaggi e Scoperte S.p.A....
Preparing points with embeddings...
Uploading 1 point(s) to collection 'fornitori'
Successfully uploaded 1 point(s) to collection 'fornitori'
Collection 'fornitori' now has 4 points
Processing DigitArt Milano S.r.l....
Preparing points with embeddings...
Uploading 1 point(s) to collection 'fornitori'
Successfully uploaded 1 

In [4]:
client = QdrantClient(
    url=os.getenv("QDRANT_URL")
)

In [13]:
    points, _ = client.scroll(
        collection_name="fornitori",
        with_payload=True,
        with_vectors=True,
    )

In [16]:
        results = client.search(
            collection_name="fornitori",
            query=query,
            limit=limit,
            with_payload=True,
            with_vectors=True,
        )

NameError: name 'query' is not defined

In [10]:
points

([Record(id='06a6c8d3-0b4a-438f-96f3-c64d6a9d9aa3', payload={'text': 'Produce e distribuisce alimenti biologici a km zero; offre servizio catering per eventi privati e aziendali, delivery di box stagionali, consulenza per riduzione degli sprechi alimentari e progetti di educazione nutrizionale nelle scuole.', 'metadata': {'page_numbers': None, 'name': 'BioCucina Italiana S.r.l.'}}, vector={'sparse': SparseVector(indices=[6913142, 172652411, 203988385, 330588669, 458027949, 503831187, 643311084, 750573363, 886597579, 901626770, 994144303, 1150976095, 1189096737, 1198476171, 1262154878, 1270919444, 1371189233, 1481838388, 1637262427, 1643915203, 1650005478, 1670835917, 1680306374, 1692081639, 1701593959, 1882043270, 1888164627, 2006536704], values=[1.5575222, 1.5575222, 1.5575222, 1.8238342, 1.5575222, 1.5575222, 1.5575222, 1.5575222, 1.5575222, 1.5575222, 1.5575222, 1.5575222, 1.5575222, 1.5575222, 1.5575222, 1.5575222, 1.5575222, 1.5575222, 1.5575222, 1.5575222, 1.5575222, 1.5575222, 1

In [15]:
all_points = []

# Use scroll to paginate through the collection
scroll_offset = None
while True:
    points, scroll_offset = client.scroll(
        collection_name="fornitori",
        limit=100,  # Adjust batch size as needed
        offset=scroll_offset,
        with_payload=True,
        with_vectors=True,
    )

    all_points.extend(points)

    if scroll_offset is None:
        break  # No more points

In [12]:
collection_name = "fornitori"

dict_keys(['sparse', 'colbertv2.0', 'dense'])

In [ ]:
# 2. Run your multi-stage query
resp = client.query_points(
    collection_name=collection_name,
    prefetch=[
        {"query": all_points[0].vector["dense"],  "using": "dense",   "limit": 5},
        {"query": all_points[0].vector["sparse"], "using": "sparse",  "limit": 5},
    ],
    query=all_points[0].vector["colbertv2.0"],
    using="colbertv2.0",
    with_payload=True,
    with_vectors=True,
    limit=5,
)

# 3. Extract results
results = resp.points  # List[ScoredPoint]

# 4. Determine number of sub-vectors for normalization
first_vec = all_points[0].vector["colbertv2.0"]
num_subvectors = len(first_vec) if isinstance(first_vec, list) else 1



# Assuming 'results' is a list of ScoredPoint objects from your query response
query_length = len(all_points[0].vector["colbertv2.0"])

for p in results:
    raw_score = p.score
    normalized_score = raw_score / query_length
    normalized_score = max(0.0, min(1.0, normalized_score))  # Optional: clamp to [0, 1]

    print(f"ID: {p.id}")
    print(f"  Raw score:        {raw_score:.4f}")
    print(f"  Normalized score: {normalized_score:.4f}")
    print(f"  Payload:          {p.payload}")
    print("---")


ID: 2f1c24ff-2579-4ad2-bc1e-b29fce84afab
  Raw score:        465.0000
  Normalized score: 0.9082
  Payload:          {'text': 'La descrizione del servizio è la seguente:\n\nIl servizio di manutenzione delle reti radio di Regione Lombardia comprende la manutenzione preventiva e correttiva per il mantenimento nelle migliori condizioni di efficienza delle reti radio regionali, tra cui:\n\n* La dorsale pluricanale regionale denominata "Alta Frequenza" (Anello nord e anello sud)\n* Le reti radio ISO frequenziali ed apparati ricetrasmittenti terminali del servizio antincendio boschivo e di protezione civile\n* I sistemi di comunicazione mobili installati sui CTM-R, CTM-P e Sala Operativa mobile (SOM), costituiti da ripetitori mobili DMR in configurazione master e satelliti, BST Tetra e connettività satellitare Tooway, Starlink e LTE 4G\n* Gli apparati terminali, portatili, veicolari e fissi in tecnologia DMR, TETRA e aeronautici\n* Il sistema di trouble ticketing\n* La gestione dei servizi a

In [20]:
while True:
    points, scroll_offset = client.scroll(
        collection_name="fornitori",
        limit=100,  # Adjust batch size as needed
        offset=scroll_offset,
        with_payload=True,
        with_vectors=True,
    )

    all_points.extend(points)

    if scroll_offset is None:
        break  # No more points

data = []
for point in all_points:
    resp = client.query_points(
        collection_name="bandi",
        prefetch=[
            {"query": point.vector["dense"], "using": "dense", "limit": 5},
            {"query": point.vector["sparse"], "using": "sparse", "limit": 5},
        ],
        query=point.vector["colbertv2.0"],
        using="colbertv2.0",
        with_payload=True,
        with_vectors=True,
        limit=5,
    )
    name_fornitori = point.payload.get("metadata", {}).get("name", "Unknown")
    query_length = len(point.vector["colbertv2.0"])
    for p in resp.points:
        raw_score = p.score
        normalized_score = raw_score / query_length
        normalized_score = max(
            0.0, min(1.0, normalized_score)
        )  # Optional: clamp to [0, 1]
        name_bandi = p.payload.get("metadata", {}).get("name", "Unknown")
        data.append([name_fornitori, name_bandi, normalized_score])


In [1]:
    points, scroll_offset = client.scroll(
        collection_name="fornitori",
        limit=100,  # Adjust batch size as needed
        offset=scroll_offset,
        with_payload=True,
        with_vectors=True,
    )

NameError: name 'client' is not defined

In [22]:
import pandas as pd

In [24]:
test = pd.DataFrame(
    data, columns=["Fornitori", "Bandi", "Normalized Score"]
)

In [28]:
test.pivot_table(index="Fornitori", columns="Bandi", values="Normalized Score")

Bandi,./data/silver/FEC 2-2025 Scheda tecnica.pdf,./data/silver/GECA 4_2025_Scheda Tecnica.pdf,./data/silver/Scheda tecnica gara Gestione Tremaglia.pdf,./data/silver/Scheda_Tecnica_FEC21_PIF_BassaPianura.pdf,./data/silver/scheda tecnica.pdf
Fornitori,,,,,
Assistenza Tecnica Puglia S.c.p.A.,0.665266,0.729799,0.660503,0.674515,0.644162
BioCucina Italiana S.r.l.,0.633426,0.617454,0.641008,0.617623,0.623332
DigitArt Milano S.r.l.,0.520639,0.535985,0.526502,0.549942,0.523330
Gestione Reti Lombardia S.r.l.,0.680963,0.639510,0.676177,0.647296,0.727647
Viaggi e Scoperte S.p.A.,0.606564,0.599477,0.616100,0.604892,0.615350


In [ ]:
all_points[0]

Record(id='2f1c24ff-2579-4ad2-bc1e-b29fce84afab', payload={'text': 'La descrizione del servizio è la seguente:\n\nIl servizio di manutenzione delle reti radio di Regione Lombardia comprende la manutenzione preventiva e correttiva per il mantenimento nelle migliori condizioni di efficienza delle reti radio regionali, tra cui:\n\n* La dorsale pluricanale regionale denominata "Alta Frequenza" (Anello nord e anello sud)\n* Le reti radio ISO frequenziali ed apparati ricetrasmittenti terminali del servizio antincendio boschivo e di protezione civile\n* I sistemi di comunicazione mobili installati sui CTM-R, CTM-P e Sala Operativa mobile (SOM), costituiti da ripetitori mobili DMR in configurazione master e satelliti, BST Tetra e connettività satellitare Tooway, Starlink e LTE 4G\n* Gli apparati terminali, portatili, veicolari e fissi in tecnologia DMR, TETRA e aeronautici\n* Il sistema di trouble ticketing\n* La gestione dei servizi aggiuntivi (setup sistemi radio "nomadici", Drive test, Tras

In [ ]:
ok.get("points", [])[0]

{'id': '2f1c24ff-2579-4ad2-bc1e-b29fce84afab',
 'version': 4,
 'score': 465.0,
 'payload': {'text': 'La descrizione del servizio è la seguente:\n\nIl servizio di manutenzione delle reti radio di Regione Lombardia comprende la manutenzione preventiva e correttiva per il mantenimento nelle migliori condizioni di efficienza delle reti radio regionali, tra cui:\n\n* La dorsale pluricanale regionale denominata "Alta Frequenza" (Anello nord e anello sud)\n* Le reti radio ISO frequenziali ed apparati ricetrasmittenti terminali del servizio antincendio boschivo e di protezione civile\n* I sistemi di comunicazione mobili installati sui CTM-R, CTM-P e Sala Operativa mobile (SOM), costituiti da ripetitori mobili DMR in configurazione master e satelliti, BST Tetra e connettività satellitare Tooway, Starlink e LTE 4G\n* Gli apparati terminali, portatili, veicolari e fissi in tecnologia DMR, TETRA e aeronautici\n* Il sistema di trouble ticketing\n* La gestione dei servizi aggiuntivi (setup sistemi r

In [ ]:
ok = search_result.model_dump()

In [ ]:
ok.get("points", [])[0]["score"]

465.0

In [ ]:

import numpy as np

In [ ]:
import numpy as np

def cosine_similarity(a: np.ndarray, b: np.ndarray) -> float:
    """
    Compute the cosine similarity between two 1-D NumPy arrays.

    Returns a value between -1 and 1, or 0 to 1 for non-negative vectors.
    """
    # Ensure inputs are 1-D and same length
    if a.ndim != 1 or b.ndim != 1 or a.shape != b.shape:
        raise ValueError("Vectors must be 1-D and of the same shape")

    dot = np.dot(a, b)
    norm_a = np.linalg.norm(a)
    norm_b = np.linalg.norm(b)

    if norm_a == 0 or norm_b == 0:
        # return 0 to avoid division by zero; define as no similarity
        return 0.0

    return dot / (norm_a * norm_b)


In [ ]:
# Ensure both vectors are 1-D and have the same shape
vec1 = np.array(ok.get("points", [])[0]["vector"]["colbertv2.0"][0]).flatten()
vec2 = np.array(ok.get("points", [])[0]["vector"]["colbertv2.0"][0]).flatten()

# Check shapes for debugging
print("vec1 shape:", vec1.shape)
print("vec2 shape:", vec2.shape)

# Use only if shapes match
if vec1.shape == vec2.shape:
    result = cosine_similarity(vec1, vec2)
    print("Cosine similarity:", result)
else:
    print("Shape mismatch:", vec1.shape, vec2.shape)

vec1 shape: (128,)
vec2 shape: (128,)
Cosine similarity: 1.0000000000000002


In [ ]:
ok.get("points", [])[0]["vector"]["colbertv2.0"][0]

In [ ]:
len(ok.get("points", [])[0]["vector"]["colbertv2.0"][0])

128

In [ ]:
print(search_result.model_dump())

{'points': [{'id': '6f73a03e-bd03-48c6-9647-608aadcdcabe',
   'version': 4,
   'score': 465.0,
   'payload': {'text': 'La descrizione del servizio è la seguente:\n\nIl servizio di manutenzione delle reti radio di Regione Lombardia comprende la manutenzione preventiva e correttiva per il mantenimento nelle migliori condizioni di efficienza delle reti radio regionali, tra cui:\n\n* La dorsale pluricanale regionale denominata "Alta Frequenza" (Anello nord e anello sud)\n* Le reti radio ISO frequenziali ed apparati ricetrasmittenti terminali del servizio antincendio boschivo e di protezione civile\n* I sistemi di comunicazione mobili installati sui CTM-R, CTM-P e Sala Operativa mobile (SOM), costituiti da ripetitori mobili DMR in configurazione master e satelliti, BST Tetra e connettività satellitare Tooway, Starlink e LTE 4G\n* Gli apparati terminali, portatili, veicolari e fissi in tecnologia DMR, TETRA e aeronautici\n* Il sistema di trouble ticketing\n* La gestione dei servizi aggiuntiv